In [ ]:
"""
El preprocesado implementado en este codigo se realiza a traves de la libreria nipype. Nipype es un wrapper
que permite utilizar las funciones de diferentes softwares de resonacia magnetica (FSL, ANTs, FreeSurfer, AFNI...)
directamente en Python al mismo tiempo. Para poder utilizar las funciones ES NECESARIO TENER INSTALADOS DICHOS SOFTWARES EN TU ORDENADOR.
Un gran tutorial sobre nipype y como instalar dichos softwares puede encontrarse en:
https://miykael.github.io/nipype-beginner-s-guide/index.html
https://miykael.github.io/nipype_tutorial/

The preprocessing implemented in this code is mainly done with nipype library. Nipype is a wrapper which allows
to use functions from different magnetic resonance imaging softwares (FSL, ANTS, FreeSurfer, AFNI...) directly in
Python and at the same time. In order to use these functions IT IS MANDATORY TO HAVE THESE SOFTWARES INSTALLED IN YOUR COMPUTER.
A great tutorial on nipype and how to install these software can be found in:
https://miykael.github.io/nipype-beginner-s-guide/index.html
https://miykael.github.io/nipype_tutorial/

"""

In [ ]:
"""
Importar dependencias
Import modules
"""

from nipype.interfaces.fsl import BET, FLIRT, ImageMaths, ImageStats
import nipype.interfaces.spm as spm
from nipype.interfaces.utility import IdentityInterface
import nibabel as nib
from nibabel.processing import resample_to_output
from nipype.interfaces.fsl import Reorient2Std
import numpy as np
from nipype.interfaces.ants import N4BiasFieldCorrection
from nipype.interfaces.utility import Function
from os.path import join as opj
from nibabel.testing import data_path
from nipype.interfaces.slicer.filtering import histogrammatching
from nipype.pipeline.engine import Workflow, Node, MapNode
import nipype.interfaces.io as nio
import multiprocessing as MultiProc
from nipype.algorithms.misc import Gunzip


# Sin estos paquetes aparece un warning al crear el workflow
# without these packages a warning pops up while creating the workflow
from scipy.ndimage import measurements
from scipy.ndimage import zoom

In [ ]:
"""
Asocia el codigo con el path donde estas trabajando (experiment_dir), donde encontrar los datos (data_dir) y la
lista de imagenes dentro del data_dir
Link the code to the path where you are working (experiment_dir), where to find the data (data_dir) and the list of
images inside the data_dir
"""

experiment_dir = "PATH"
data_dir = opj("PATH/data") #Mas sencillo si los datos se encuentran dentro del experiment_dir / It's easier if data_dir is inside experimet_dir

image_list = ["imagen001", "imagen002", "imagen003", "imagen004",
              "imagen005", "imagen006", "imagen007", "imagen008",
              "imagen009", "imagen010"]
#Nombres de las carpetas que contienen los archivos .nii / Names of the folder which contains the .nii files
#Los nombres de las imagenes deben tener los mismos nombres (imagen001/struct.nii, imagen002/struct.nii,...)
#The names of the images must have the same names (image001/struct.nii, image002/struct.nii,...)

In [ ]:
"""
Especificar las variables de las funciones que se van a utilizar y el nombre de la estructura de carpetas del output
Specify the arguments of the functions to use and the names of the folder structure of the output
"""

#Para la extraccion del cerebro con BET de FSL
# For brain extraction with FSL's BET 
frac = 0.1
reduce_bias = True

#Registro lineal con FLIRT de FSL
#Linear Registration with FSL's FLIRT

template = "path_to_template" #Por ejemplo MNI152 / for example MNI152

#Files
output_dir = "folder_name_for_final_image" #Carpeta para la ultima imagen
working_dir = "folder_name_for_intermediate_images" #Carpeta para las imagenes intermedias

In [ ]:
"""
Especificar los nodos que contienen las funciones del preprocesado
Specify the nodes which contains the preprocessing functions

"""

# Extraer archivos comprimidos .nii.gz a .nii pues N4 necesita archivos descomprimidos como input
# Extract compressed files .nii.gz to .nii because N4 require uncompressed files as input
gunzip = Node(Gunzip(), name="gunzip")

#Correccion de la inhomogeneidad con N4 de ANTs
#Bias correction with ANTs' N4
n4 = Node(N4BiasFieldCorrection(), name = "bias_correction")

#Extraccion del cerebro con BET
#Brain extraction
bet = Node(BET(), name = "bet")
bet.inputs.frac = frac
bet.inputs.reduce_bias = reduce_bias

#Reorientar para el registro
#Reorient before registration
reorient = Node(Reorient2Std(), name = "reorient")

#Registro lineal con FLIRT
#Linear registration FLIRT
flirt = Node(FLIRT(), name = "flirt")

flirt.inputs.reference = template

#Normalizacion de la intensidad mediante min-max, la opción op_string='-r' elimina los valores extremos de la normalizacion, -R los mantiene
#Intensity min max normalization, argument op_string='-r' remove extreme values from the normalization, -R keeps them.

fslstats = MapNode(interface = ImageStats(op_string='-r'),
                          name = 'fslstats', iterfield=['in_file'])
def min_max(in_stat):
    min_val, max_val = in_stat
    return '-sub %s -div %s' % (min_val, (max_val-min_val))

stat_to_op_string = MapNode(interface=Function(input_names=['in_stat'],
                                                output_names=['op_string'],
                                                function=min_max),
                                   name='stat_to_op_string', iterfield=['in_stat'])

fslmaths = MapNode(interface = ImageMaths(),
                          name = 'fslmaths', iterfield=['in_file', 'op_string'])


In [ ]:
"""
Especificar los nodos de input y output
Specify input and output nodes
"""

#Recoge las imagenes como input con el nombre de variable "image_id"
#Picks the images as inputs with the variable name "image_id"
infosource = Node(IdentityInterface(fields = ["image_id"]), 
                    name = "infosource")

#Transforma las carpetas que contienen las images recogidas en image_list en iterables "image_id"
#Transforms the folder that contains the images in image_list into iterables "image_id" 
infosource.iterables = [('image_id', image_list)]

#En caso de que se utilicen más de un tipo de imagen (T1, T2, funcional...), recoge cada tipo en un diccionario, en este caso concreto solo hay una estructural, bajo la key:anat
#In case that more than one type of image are used (T1, T2, functional...), it picks each kind in a dictionary, in this particular case only one kind is used under the key:anat
templates = {'anat': 'data_dir/{image_id}/name_of_the_images.nii.gz'}
selectfiles = Node(nio.SelectFiles(templates,
                                  base_directory = experiment_dir), name="selectfiles")


datasink = Node(nio.DataSink(), name="datasink")
datasink.inputs.base_directory = "PATH_to_folder" (working_dir) # nombre de la carpeta donde guardar las imagenes intermedias / name of the folder where to save the intermediate images
datasink.inputs.container = "PATH_to_folder" (output_dir) # nombre de la carpeta donde guardar la imagen final / name of the folder where to save the final image

In [ ]:
"""
Crear workflow y determinar su directorio
Create the workflow and determinate its directories
"""

preprocess_workflow = Workflow(name = "preprocess_full_workflow")
preprocess_workflow.base_dir = opj(experiment_dir, working_dir)


In [ ]:
"""
Conectar los nodos
Conect the nodes
"""

#Escribe el input y outpu de cada nodo. Cada funcion utiliza diferentes nombres para los inputs y outputs
#Write the input and output for each node. Each function has its own name for inputs and outputs

preprocess_workflow.connect([
                    (infosource, selectfiles, [("image_id", "image_id")]),
                    (selectfiles, gunzip, [("anat", "in_file")]),
                    (gunzip, n4, [("out_file", "input_image")]),
                    (n4, reorient, [("output_image", "in_file")]),
                    (reorient, bet, [("out_file", "in_file")]),
                    (bet, flirt, [("out_file", "in_file")]),
                    (flirt, fslstats, [("out_file", "in_file")]),
                    (flirt, fslmaths, [("out_file", "in_file")]),
                    (fslstats, stat_to_op_string, [('out_stat', 'in_stat')]),
                    (stat_to_op_string, fslmaths, [('op_string', 'op_string')]),
                    (fslmaths, datasink, [("out_file", "resultados")]),
                    ])

In [ ]:
"""
Ejecuta el workflow
Execute the workflow
"""
#Puede hacerse en serie o en paralelo. En este caso es en paralelo utiliza 6 nucleos del procesador
#This can be donde either secuencially or in parallel. In this case is done in parallel with 6 cores of the CPU.

preprocess_workflow.run('MultiProc', plugin_args={'n_procs': 6})